### MongoDbOperations Class

This class is used to create MongoDB Database, Collection and Insert Multiple csv files to DB. It can also be used to fetch records from MongoDB and save it as .csv file. Records can be saved in different formats by changing the code. Mainly helpful if you have bulk of files to push to MongoDB or fetch all records from MongoDB.

In [9]:
from pymongo import MongoClient
import pandas as pd
import json
import os


class MongoDBOperations:
    
    """
    Description: This class is used to create MongoDB Database, Collection and Insert Multiple csv files to DB.
                Also, get records from MongoDB and save it as .csv file. Records can also be saved in different 
                formats by changing the code.
    Output: Returns client object

    """

    def __init__(self):
        
        
        # Give a specified file path where your data is stored 
        self.goodFilePath = "<file_path>"
        
        # Provide URI from MongoDB Atlas along with your username and password, example as below:
        self.url = "mongodb+srv://<username>:<password>@cluster0.qjde0.mongodb.net/myFirstDatabase?retryWrites=true&w=majority"
        

    def getMongoClient(self):

        """
        Description: This method sets the MongoDb client object.
        Output: Returns client object

        """

        self.client = MongoClient(self.url)
        return self.client

    def isDbPresent(self, db_name):

        """
        Description: This method checks if DB is present.
        Output: Returns True if exists, False otherwise.

        """

        client = self.getMongoClient()

        if db_name in client.list_database_names():
            return True
        else:
            return False

    def CreateDB(self, db_name):

        """
        Description: This method creates DB if not present. If present, deletes the existing DB and creates a new one.
        Output: Returns database created

        """

        check_db = self.isDbPresent(db_name)
        client = self.getMongoClient()

        try:

            if check_db == False:
                database = client[db_name]
                client.close()
                return database

            else:
                client.drop_database(db_name)
                database = client[db_name]
                client.close()
                return database

        except Exception as e:
            return e

    def getDB(self, db_name):

        """
        Description: This method gets the database object.
        Output: Returns database object

        """
        client = self.getMongoClient()
        client.close()

        try:
            return client[db_name]

        except Exception as e:
            return e

    def CreateCollection(self, db_name, collection_name):

        """
        Description: This method creates database and then the collection
        Output: Returns collection created

        """

        database = self.CreateDB(db_name)

        try:
            collection = database[collection_name]
            return collection

        except Exception as e:
            return e

    def getCollection(self, db_name, collection_name):

        """
        Description: This method calls the databse object and returns collection object
        Output: Returns collection object

        """
        try:
            database = self.getDB(db_name)
            return database[collection_name]

        except Exception as e:
            raise Exception(f"(getCollection): Failed to get the database list.")

    def InsertDataToCollection(self, db_name, collection_name):

        """
        Description: This method creates database and collection to insert multiple csv files stored
        in a particular folder to MongoDB
        Output: None

        """
        good_file_path = self.goodFilePath
        
        # listing the csv files in a directory
        csv_files = os.listdir(good_file_path)
        
        collection = self.CreateCollection(db_name, collection_name)

        try:
            # Code to upload n number of csv files to DB
            
            for files in csv_files:
                df = pd.read_csv(good_file_path+files)
                data = df.to_dict(orient='records')
                collection.insert_many(data)

        except Exception as e:
            return e

    def GetDataFromCollecttion(self, db_name, collection_name):

        """
        Description: This method gets all records from a collection and saves it as csv file
        Output: Saves records in csv file
        """

        collection = self.getCollection(db_name, collection_name)
        # Provide the name of folder where you want to save the file
        self.folder_name = 'Name_of_Folder_to_save_file'
        # Provide the name of csv file where the records needs to be saved
        self.file_name = 'InputFile.csv'

        try:
            result = collection.find({})
            df = pd.DataFrame(result)
            # Creating a folder if it doesn't exist to save file
            if not os.path.isdir(self.folder_name):
                os.makedirs(self.folder_name)
            
            # Saving the file as csv
            df.to_csv(self.folder_name+self.file_name, mode='w')

        except Exception as e:
            return e


